In [1]:
import pandas as pd
from textblob import TextBlob
from newspaper import Article
from datetime import timedelta
import spacy
from collections import Counter
from string import punctuation

# from googlesearch import search


import sys
sys.path.append('../')
import utility_functions



In [2]:
nlp = spacy.load("en_core_web_md")

In [3]:
reference = pd.read_csv('../data/processed/input-data.csv',parse_dates=['time_parsed'])

In [4]:
len(reference)

2921

# Starting Selenium Server

In [5]:
# Selenium Imports 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains

# Utility Imports
import copy
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import re 
from pathlib import Path
import os
import time
from datetime import date
from datetime import timedelta
from datetime import datetime
import dateutil.parser as parser
import urllib.request
import random
from dateutil.parser import parse
cwd = os.getcwd()

In [6]:
# Starting the selenium webserver and getting whatsapp 
userprofile = 'samarth'
options = webdriver.ChromeOptions()
# options.add_argument(f"user-data-dir=/Users/flipflops/Library/Application Support/Google/Chrome/Profile 3")
driver = webdriver.Chrome(ChromeDriverManager().install() , options=options)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/caochao/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60/chromedriver] found in cache
/var/folders/c8/z7rkgj2x1fz_257xzx4c_vdh0000gn/T/ipykernel_743/813679169.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install() , options=options)


In [ ]:
# Batch n 

BATCH_NO = 0 
SCRAPED_BY = 'sam'

startIndex = 2500
endIndex = 2900

print(f'Getting Dataset From {startIndex} to {endIndex} ')



for index in range(startIndex,endIndex):
    print(f'-------------{index}')
    batch = []
    current_ref = reference.iloc[index]
    title = current_ref['title']
    if type(title)== float:
        continue

    print(f'debug -- title: {title}')
    titleHotwords = utility_functions.get_hotwords(title, nlp)
    uid = current_ref['uid']
    modifiedHotwords = utility_functions.add_numerics(title,titleHotwords)
    timeFrom = str(current_ref['time_parsed'].date()) # Date at the time of reporting
    timeTo = str((current_ref['time_parsed'] + timedelta(days=2)).date())  # Date 2 days from the date of reporting 
    
    SEARCHTERM = ' '.join(modifiedHotwords) + ' after:' +timeFrom + ' before:' + timeTo
    print(f'Searching for\n----- {SEARCHTERM}')
    
    searches = 0 
    
    
    notSearchClass = 'gLFyf gsfi'
    searchButton = driver.find_elements(By.XPATH,f"//input[@class='{notSearchClass}']")
    searchButton[0].clear()
    time.sleep(2)
    searchButton[0].send_keys(SEARCHTERM)
    time.sleep(3)
    searchButton[0].send_keys(Keys.ENTER)
    time.sleep(8)
    innerSearchClass = 'yuRUbf'
    searchResults = driver.find_elements(By.XPATH,f"//div[@class='{innerSearchClass}']/a")
    
    if len(searchResults) == 0 :
        continue
        
    searchResults[0].get_attribute('href')
    
    

    
    
    for searchResult in [x.get_attribute('href') for x in searchResults]:
        
        if 'liveuamap.com' in searchResult or 'wikipedia.org' in searchResult :
            continue

        
        print(searchResult,end = ' ---')
        
        try:

            article_parse = Article(searchResult)
            article_parse.download()
            article_parse.parse()
            article_author = article_parse.authors
            article_date = article_parse.publish_date
            article_text = article_parse.text

            batch.append({
                'uid' : uid,
                'article_author' : article_author,
                'article_date' : article_date,
                'article_text' : article_text,
                'article_url' : searchResult
            })
            searches = searches + 1 
            print('Done')
        except Exception:
            print('Failed:Skipping')
        if searches >10:
            break
    
    pd.DataFrame(batch).to_csv(f'../data/processed/articles/{index}.csv')
    time.sleep(15)

            
            
    

Getting Dataset From 2500 to 2900 
-------------2500
debug -- title: Explosions at Saltivka district in Kharkiv
Searching for
----- explosions saltivka district kharkiv after:2022-04-02 before:2022-04-04
https://www.independent.co.uk/news/world/europe/ukraine-kharkiv-residents-russia-bombing-b2049901.html ---Done
https://nationworldnews.com/russia-sends-a-message-to-all-of-ukraine-by-hitting-these-civilian-areas-in-this-city/ ---Done
https://maidan.org.ua/en/2022/04/kharkiv-khroniky-ataky-na-misto-den-37-y-01-04-2022/ ---Done
https://twitter.com/2022kharkiv?lang=en ---Done
https://ndtvweb.com/we-have-now-survived-in-bombed-out-kharkiv-residents-modify-to-life-underground/ ---Done
https://reliefweb.int/report/ukraine/ukraine-humanitarian-impact-situation-report-1200-pm-eet-4-april-2022 ---Done
-------------2501
debug -- title: Almost 300 people buried in 'mass grave' in Bucha outside Kyiv: mayor
Number 300 not found in hotwords, adding
Searching for
----- people mass grave bucha kyiv ma

https://reliefweb.int/report/ukraine/ukraine-humanitarian-impact-situation-report-1200-pm-eet-4-april-2022 ---Done
https://www.bbc.com/news/live/world-europe-60949706/page/12 ---Done
https://www.washingtonpost.com/national-security/2022/04/03/ukraine-russia-zelensky/ ---Done
-------------2509
debug -- title: Russian strike against infrastructure in Myrhorod with 3 missiles, preliminary no casualties - head of regional administration
Number 3 not found in hotwords, adding
Searching for
----- russian strike infrastructure myrhorod missiles preliminary casualties head regional administration 3 after:2022-04-02 before:2022-04-04
https://www.jpost.com/breaking-news/article-703003 ---Failed:Skipping
https://rubryka.com/en/article/38-day-war/ ---Done
https://www.khaleejtimes.com/world/live-russia-ukraine-crisis-us-pentagon-commits-extra-300-million-in-security-aid-to-ukraine ---Done
https://socialgrep.com/search?query=brovary ---Done
https://socialgrep.com/index.php/search?query=american%C2%A

https://hindustannewshub.com/world-news/putin-talks-emmanuel-macron-putin-bluntly-in-talks-with-macron-the-goals-of-the-russian-special-operations-in-ukraine-will-be-met-at-any-cost/ ---Done
https://www.thedenverchannel.com/news/national/putin-reportedly-strikes-defiant-tone-in-call-with-macron ---Done
-------------2517
debug -- title: White House is asking Congress for $10 billion in emergency funding to boost its response to invasion of Ukraine—money that will be used to address humanitarian crisis as well as assist its defense against Russia
Searching for
----- white house congress emergency funding response invasion ukraine money humanitarian crisis defense russia after:2022-03-03 before:2022-03-05
https://www.whitehouse.gov/omb/briefing-room/2022/03/03/meeting-urgent-needs/ ---Done
https://www.nytimes.com/2022/03/03/us/politics/white-house-ukraine-pandemic-aid.html ---Done
https://www.bloombergquint.com/politics/biden-asks-congress-for-32-5-billion-in-ukraine-covid-funds ---Done
h

https://www.usip.org/publications/2022/03/china-not-russia-taiwan-not-ukraine ---Done
https://www.jstor.org/stable/community.32147092 ---Failed:Skipping
https://www.jstor.org/stable/community.32149611 ---Failed:Skipping
https://www.focaalblog.com/tag/political-anthropology/ ---Done
https://issuu.com/laprensalatina/docs/lpl_03.06.2022?fr=sMDgxZDQ3MDYyMTQ ---Done
https://www.congress.gov/88/crecb/1963/03/04/GPO-CRECB-1963-pt3-7-1.pdf ---Done


/var/folders/c8/z7rkgj2x1fz_257xzx4c_vdh0000gn/T/ipykernel_743/948238369.py:85: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  pd.DataFrame(batch).to_csv(f'../data/processed/articles/{index}.csv')


...

https://theaviationist.com/2022/03/03/a-10-ukraine/ ---Done
https://www.independent.co.uk/news/world/europe/russia-military-convoy-kyiv-uk-b2028436.html ---Done
https://www.cnn.com/2022/03/03/europe/russian-convoy-stalled-outside-kyiv-intl/index.html ---Done
-------------2562
-------------2563
debug -- title: Consequences of Russian shelling of Izyum
Searching for
----- consequences russian shelling izyum after:2022-03-03 before:2022-03-05
https://www.amnesty.org/en/wp-content/uploads/2022/03/EUR5053822022ENGLISH.pdf ---Done
https://www.iphronline.org/ru-war-crimes-in-ua-4.html ---Done
https://www.atlanticcouncil.org/blogs/new-atlanticist/russian-war-report-global-crackdown-on-russian-state-affiliated-media-continues-as-russia-blocks-facebook/ ---Done
https://www.indiatoday.in/world/russia-ukraine-war/story/war-changed-ukraine-one-week-russia-shelling-kyiv-kherson-refugee-crisis-1920109-2022-03-03 ---Done
https://euromaidanpress.com/2022/03/03/war-in-ukraine-day-7-russia-escalates-air-

https://www.reddit.com/r/CombatFootage/comments/t5wuqn/some_rumarked_vehicles_get_disposed_under_mykolaiv/ ---Done
https://www.reddit.com/r/CombatFootage/comments/t6uhde/battle_of_grozny_19941995/ ---Done
https://chicagoboyz.net/archives/67331.html ---Done
-------------2572
debug -- title: The @StateDept issued a strong statement to condemn Russia's "full assault on media freedom and the truth."Independent outlets Radio Ekho Moskvy and Dozhd TV were restricted. RosKomNadzor threatened to block @GolosAmeriki if it does not take down reporting on Russia's invasion
Searching for
----- @statedept strong statement russia assault media freedom truth "independent outlets radio ekho moskvy dozhd tv roskomnadzor @golosameriki russia invasion after:2022-03-02 before:2022-03-04
https://www.voanews.com/a/latest-developments-in-ukraine-march-2-/6466013.html ---Done
-------------2573
debug -- title: Military echelon in Tyumen
Searching for
----- military echelon tyumen after:2022-03-02 before:2022-0

https://www.maritime-executive.com/article/fears-rise-with-reports-bangladeshi-engineer-killed-in-rocket-attack ---Done
https://www.fleetmon.com/maritime-news/2022/37481/bangladeshi-bulk-carrier-hit-russian-navy-missile-/ ---Done
https://gcaptain.com/bangladeshi-ship-hit-by-missile-at-ukrainian-port-reports/ ---Done
https://www.seanews.com.tr/russians-hit-the-anchored-bangladeshi-ship-banglar-samriddhi-with-a-missile-1-dead/192641/ ---Done
https://www.ukrinform.net/rubric-ato/3418371-russian-missile-hits-foreign-vessel-in-mykolaiv-port.html ---Done
https://www.perild.com/2022/03/03/the-moment-of-a-missile-attack-on-a-ship-in-the-port-of-nikolaev-was-caught-on-video/ ---Done
https://newsreadonline.com/there-is-a-dead-person-on-the-banglar-samriddhi-ship-attacked-by-russian-missile-ministry-of-infrastructure/ ---Done
-------------2581
debug -- title: President Biden:Putin thought the West and NATO wouldn't respond — and he thought he could divide us at home. He was wrong. I spent countle

https://www.washingtonpost.com/world/2022/04/09/russia-ukraine-war-news-putin-live-updates/ ---Done
https://www.theguardian.com/education/2022/apr/08/ukrainian-langauge-interest-spikes-support-country-war ---Done
https://news.sky.com/story/ukraine-news-live-chemical-attack-mariupol-russia-putin-latest-12541713?postid=3693360 ---Done
-------------2587
debug -- title: "#Russia's invasion isn't just a mortal threat to #Ukraine. It's a challenge to the rules-based international system" per @SecDef "We've rushed them security assistance. We've rallied our @NATO allies. & we've reinforced @NATO's Eastern Flank with more combat power
Searching for
----- russia invasion mortal threat ukraine challenge rules international system @secdef security assistance @nato allies @nato eastern flank combat power after:2022-04-01 before:2022-04-03
https://twitter.com/pentagonpressec ---Done
-------------2588
debug -- title: 17:40 A column of 30 pieces of Russian military equipment was seen near Rechitsa, m

https://www.yahoo.com/video/israeli-pm-condemns-bucha-killings-124618434.html ---Done
https://theweek.com/russo-ukrainian-war/1012222/russias-bucha-horrors-are-pushing-india-turkey-other-studiously-neutral ---Done
https://ejpress.org/israeli-foreign-minister-yair-lapid-condemns-what-he-calls-russias-war-crimes-in-ukraine/ ---Done
https://www.statecraft.co.in/article/israel-breaks-silence-on-russia-condemns-war-crimes-in-bucha ---Done
-------------2600
debug -- title: Putin convened meeting of Russian National security council
Searching for
----- putin meeting russian national security council after:2022-03-11 before:2022-03-13


In [ ]:
!open .